拼接（需要arcpy环境）

In [1]:
import os
import glob
import arcpy

arcpy.env.overwriteOutput = True # 允许覆盖输出文件

# 设置输入输出路径
input_folder = r"D:\新建文件夹\1234\123"

# 全色波段
# 获取并校验文件列表
LH_Tif = glob.glob(os.path.join(input_folder, "*LH.tif"))
# 将文件名列表转为分号分隔的字符串
input_rasters = ";".join(sorted([os.path.abspath(f) for f in LH_Tif], reverse=True))

# 镶嵌至新栅格
arcpy.management.MosaicToNewRaster(
    input_rasters=input_rasters,
    output_location=r"D:\新建文件夹\123\Mosaic",  # 输出路径
    raster_dataset_name_with_extension="PAN.tif",
    coordinate_system_for_the_raster='PROJCS["WGS_1984_UTM_Zone_51N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    pixel_type="16_BIT_UNSIGNED",
    cellsize=None,
    number_of_bands=3,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

# RGB波段
# 获取并校验文件列表
RGB_Tif = glob.glob(os.path.join(input_folder, "*RGB.tif"))
# 将文件名列表转为分号分隔的字符串
input_rasters = ";".join(sorted([os.path.abspath(f) for f in RGB_Tif], reverse=True))

# 镶嵌至新栅格
arcpy.management.MosaicToNewRaster(
    input_rasters=input_rasters,
    output_location=r"D:\新建文件夹\123\Mosaic",  # 输出路径
    raster_dataset_name_with_extension="RGB.tif",
    coordinate_system_for_the_raster='PROJCS["WGS_1984_UTM_Zone_51N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    pixel_type="16_BIT_UNSIGNED",
    cellsize=None,
    number_of_bands=3,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

<Result 'D:\\新建文件夹\\123\\Mosaic\\RGB.tif'>

加载模块

In [1]:
import os
import xml.etree.ElementTree as ET
import rasterio
from rasterio.windows import Window
from rasterio.warp import reproject
from rasterio.enums import Resampling
import numpy as np
from tqdm import tqdm, trange
import glob
import traceback


光区识别

In [2]:
# 计算块内部像素8邻域非零像素数
def get_neighbor_count(block):
    padded = np.pad(block, pad_width=1, mode='constant', constant_values=0)
    neighbor = np.zeros_like(block, dtype=np.uint8)
    for dx in [-1,0,1]:
        for dy in [-1,0,1]:
            if dx == 0 and dy == 0:
                continue
            neighbor += padded[1+dx:1+dx+block.shape[0], 1+dy:1+dy+block.shape[1]] > 0
    return neighbor

# 分块处理光区识别
def light_area(raster_path, threshold, S, output_path, block_size=512):
    with rasterio.open(raster_path) as src:
        meta = src.meta.copy()
        meta.update(count=1)  # 输出单波段

        width = src.width
        height = src.height

        with rasterio.open(output_path, 'w', **meta) as dst:
            for row_off in trange(0, height, block_size, desc="块行进度"):
                for col_off in trange(0, width, block_size, desc="块列进度", leave=False):
                    win_width = min(block_size, width - col_off)
                    win_height = min(block_size, height - row_off)

                    # 支持边缘扩展1像素，防止邻域遗漏
                    pad_left = 1 if col_off > 0 else 0
                    pad_top = 1 if row_off > 0 else 0
                    pad_right = 1 if (col_off + win_width) < width else 0
                    pad_bottom = 1 if (row_off + win_height) < height else 0

                    read_window = Window(
                        col_off - pad_left,
                        row_off - pad_top,
                        win_width + pad_left + pad_right,
                        win_height + pad_top + pad_bottom
                    )

                    PL_block = src.read(1, window=read_window)
                    PH_block = src.read(2, window=read_window)

                    # 去暗电流噪声
                    PL_block[PL_block < threshold] = 0
                    PH_block[PH_block < threshold] = 0

                    PL_bin = (PL_block > 0).astype(np.uint8)
                    PH_bin = (PH_block > 0).astype(np.uint8)

                    PL_neighbor = get_neighbor_count(PL_bin)
                    PH_neighbor = get_neighbor_count(PH_bin)

                    PL_filtered = np.where(PL_neighbor >= S, PL_bin, 0)
                    PH_filtered = np.where(PH_neighbor >= S, PH_bin, 0)

                    # 去除pad，只保留核心块大小
                    core_PL = PL_filtered[pad_top:pad_top+win_height, pad_left:pad_left+win_width]
                    core_PH = PH_filtered[pad_top:pad_top+win_height, pad_left:pad_left+win_width]

                    intersection = np.logical_and(core_PL, core_PH).astype(np.uint8) * 255

                    # 写入对应块
                    dst_window = Window(col_off, row_off, win_width, win_height)
                    dst.write(intersection, 1, window=dst_window)
    return 0

去噪

In [4]:
# 全色波段去噪函数
def denoise_ph(Intersection, PAN, output_path):
    """
    对全色波段 (PH) 进行去噪处理。

    参数:
        Intersection (str): 光区识别结果的文件路径。
        PAN (str): 全色波段影像文件路径。
        output_path (str): 去噪后的全色波段输出路径。

    返回:
        int: 0 表示成功。
    """
    with rasterio.open(PAN) as pan_src:
        ph_data = pan_src.read(2)  # PH波段，确认索引是否正确
        ph_meta = pan_src.meta

    with rasterio.open(Intersection) as illu_src:
        illumination_data = illu_src.read(1)

    illumination_mask = (illumination_data > 0).astype(np.uint8)
    denoised_ph = ph_data * illumination_mask

    ph_meta.update({"count":1, "dtype":denoised_ph.dtype})

    with rasterio.open(output_path, "w", **ph_meta) as dest:
        dest.write(denoised_ph, 1)

    return 0

# RGB去噪
def denoise_rgb(Intersection, RGB, output_path):
    """
    对 RGB 波段影像进行去噪处理。

    参数:
        Intersection (str): 光区识别结果的文件路径。
        RGB (str): RGB 波段影像文件路径。
        output_path (str): 去噪后的 RGB 波段输出路径。

    返回:
        int: 0 表示成功。
    """
    with rasterio.open(RGB) as rgb_src:
        rgb_data = rgb_src.read()
        rgb_meta = rgb_src.meta

    with rasterio.open(Intersection) as illu_src:
        illumination_data = illu_src.read(1)
        illumination_meta = illu_src.meta

    resampled_illumination = np.zeros_like(rgb_data[0])
    reproject(
        source=illumination_data,
        destination=resampled_illumination,
        src_transform=illumination_meta['transform'],
        dst_transform=rgb_meta['transform'],
        src_crs=illumination_meta['crs'],
        dst_crs=rgb_meta['crs'],
        resampling=Resampling.nearest
    )
    illumination_mask = (resampled_illumination > 0).astype(np.uint8)

    denoised_rgb = np.zeros_like(rgb_data)
    for band in trange(rgb_data.shape[0], desc="RGB波段去噪"):
        denoised_rgb[band] = rgb_data[band] * illumination_mask

    rgb_meta.update(dtype=denoised_rgb.dtype)
    with rasterio.open(output_path, "w", **rgb_meta) as dest:
        dest.write(denoised_rgb)
    return 0
    

解析定标系数（xml）文件

In [5]:
# 解析xml文件定标系数
def parse_giu_calibration(xml_path):
    """
    解析 XML 文件中的定标系数。

    参数:
        xml_path (str): 包含 XML 文件的文件夹路径。

    返回:
        dict: 包含定标系数的字典，格式为 {波段: {gain: 值, bias: 值}}。
    """
    input_folder = xml_path
    head_files = glob.glob(os.path.join(input_folder, "*calib.xml"))

    with open(os.path.abspath(head_files[0]), 'rb') as f:
        raw_data = f.read().decode('gb2312')
        parser = ET.XMLParser(encoding='gb2312')
        tree = ET.ElementTree(ET.fromstring(raw_data, parser=parser))

    version_node = tree.find('./RADIOMETRIC_CALIBRATION/GIU/VERSION[@NO="V2.0"]')

    calibration = {}
    for element in version_node:
        tag = element.tag
        value = float(element.text)
        band_type = tag.split('_')[-1]
        coefficient_type = 'gain' if 'GAIN' in tag else 'bias'
        calibration.setdefault(band_type, {})[coefficient_type] = value

    return calibration

辐射校正

In [8]:
def radiance_cor(rgb_path, ph_path, calibration, out_rgb, out_ph, block_size=(512, 512)):
    """
    对影像进行辐射校正。

    参数:
        rgb_path (str): RGB 波段影像文件路径。
        ph_path (str): 全色波段影像文件路径。
        calibration (dict): 定标系数字典，格式为 {波段: {gain: 值, bias: 值}}。
        out_rgb (str): 校正后的 RGB 波段输出路径。
        out_ph (str): 校正后的全色波段输出路径。
        block_size (tuple): 分块大小，默认为 (512, 512)。

    返回:
        int: 0 表示成功，-1 表示失败。
    """
    try:
        # 打开 RGB 和 PH 影像文件
        with rasterio.open(rgb_path) as rgb_src, rasterio.open(ph_path) as ph_src:
            rgb_meta = rgb_src.meta
            ph_meta = ph_src.meta

            # 获取影像的宽度和高度
            rgb_width, rgb_height = rgb_src.width, rgb_src.height
            ph_width, ph_height = ph_src.width, ph_src.height

            # 更新元数据
            rgb_meta.update(dtype='float32', count=3)  # RGB 影像有 3 个波段
            ph_meta.update(dtype='float32', count=1)  # PH 影像只有 1 个波段

            # 处理 RGB 影像
            with rasterio.open(out_rgb, 'w', **rgb_meta) as rgb_dst:
                for i in range(0, rgb_height, block_size[1]):
                    for j in range(0, rgb_width, block_size[0]):
                        # 动态调整窗口大小，确保不超出影像范围
                        win_width = min(block_size[0], rgb_width - j)
                        win_height = min(block_size[1], rgb_height - i)
                        window = Window(j, i, win_width, win_height)

                        # 读取当前窗口的数据
                        rgb_data = rgb_src.read(window=window)

                        # 初始化校正后的 RGB 数据
                        corrected_rgb = np.zeros_like(rgb_data, dtype=np.float32)

                        # 对每个波段进行校正
                        for band in range(rgb_data.shape[0]):
                            band_type = ['RED', 'GREEN', 'BLUE'][band]
                            if band_type in calibration:
                                gain = calibration[band_type]['gain']
                                bias = calibration[band_type]['bias']
                                corrected_rgb[band] = (rgb_data[band] * gain) + bias

                        # 转换辐射单位
                        corrected_rgb[0] *= 294 * 100  # 红色波段
                        corrected_rgb[1] *= 106 * 100  # 绿色波段
                        corrected_rgb[2] *= 102 * 100  # 蓝色波段

                        # 写入校正后的数据
                        rgb_dst.write(corrected_rgb.astype(np.float32), window=window)

            # 处理 PH 影像
            with rasterio.open(out_ph, 'w', **ph_meta) as ph_dst:
                for i in range(0, ph_height, block_size[1]):
                    for j in range(0, ph_width, block_size[0]):
                        # 动态调整窗口大小，确保不超出影像范围
                        win_width = min(block_size[0], ph_width - j)
                        win_height = min(block_size[1], ph_height - i)
                        window = Window(j, i, win_width, win_height)

                        # 读取当前窗口的数据
                        ph_data = ph_src.read(window=window)

                        # 初始化校正后的 PH 数据
                        corrected_ph = np.zeros_like(ph_data, dtype=np.float32)

                        # 对 PH 波段进行校正
                        band_type = 'PH'  # 假设 PH 影像只有一个波段
                        if band_type in calibration:
                            gain = calibration[band_type]['gain']
                            bias = calibration[band_type]['bias']
                            corrected_ph[0] = (ph_data[0] * gain) + bias

                        # 转换辐射单位
                        corrected_ph[0] *= 466 * 100  # PH 波段

                        # 写入校正后的数据
                        ph_dst.write(corrected_ph.astype(np.float32), window=window)

        print("辐射校正完成")
        return 0  # 返回成功标识

    except Exception as e:
        print(f"辐射校正失败: {e}")
        traceback.print_exc()  # 打印完整堆栈信息，方便调试
        return -1

主函数

In [9]:
import os
from tqdm import tqdm

if __name__ == "__main__":
    # 路径自定义
    PAN_PATH = r"D:\新建文件夹\123\Mosaic\PAN.tif"  # 全色波段路径
    RGB_PATH = r"D:\新建文件夹\123\Mosaic\RGB.tif"  # RGB波段路径
    XML_FOLDER = r"D:\新建文件夹\1234\123"          # 定标系数XML文件夹路径
    OUTPUT_DIR = r"D:\新建文件夹\123\处理过程"      # 处理过程输出路径
    RESULT_DIR = r"D:\新建文件夹\123\处理结果"      # 处理结果输出路径

    # 创建文件夹
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(RESULT_DIR, exist_ok=True)

    # 路径检查函数
    def check_path(path, is_file=False):
        if is_file and not os.path.isfile(path):
            raise FileNotFoundError(f"文件不存在: {path}")
        elif not is_file and not os.path.isdir(path):
            raise FileNotFoundError(f"文件夹不存在: {path}")

    # 检查路径是否存在
    try:
        check_path(PAN_PATH, is_file=True)
        check_path(RGB_PATH, is_file=True)
        check_path(XML_FOLDER)
    except FileNotFoundError as e:
        print(e)
        exit(1)

    # 输出文件路径定义
    LIGHT_AREA_PH = os.path.join(OUTPUT_DIR, "Light_area_ph.tif")
    DENOISED_PH = os.path.join(OUTPUT_DIR, "denoised_ph.tif")
    DENOISED_RGB = os.path.join(OUTPUT_DIR, "denoised_rgb.tif")
    RESULT_RGB = os.path.join(RESULT_DIR, "denoised_rgb.tif")
    RESULT_PH = os.path.join(RESULT_DIR, "denoised_ph.tif")

    # 全局参数设置
    LIGHT_AREA_THRESHOLD = 3
    LIGHT_AREA_NEIGHBOR = 1

    # 任务列表定义
    tasks = [
        ("光区识别", lambda: light_area(PAN_PATH, LIGHT_AREA_THRESHOLD, LIGHT_AREA_NEIGHBOR, LIGHT_AREA_PH)),
        ("PH波段去噪", lambda: denoise_ph(LIGHT_AREA_PH, PAN_PATH, DENOISED_PH)),
        ("RGB波段去噪", lambda: denoise_rgb(LIGHT_AREA_PH, RGB_PATH, DENOISED_RGB)),
        ("解析定标系数", lambda: parse_giu_calibration(XML_FOLDER)),
        ("辐射校正", lambda: radiance_cor(DENOISED_RGB, DENOISED_PH, calib_data, RESULT_RGB, RESULT_PH)),
    ]

    # 执行任务循环
    calib_data = None
    for desc, func in tqdm(tasks, desc="整体流程进度"):
        try:
            print(f"开始任务: {desc}")
            result = func()
            if desc == "解析定标系数":
                calib_data = result
                if calib_data is None:
                    raise ValueError("定标系数解析失败，返回为空！")
                elif not isinstance(calib_data, (dict, list, tuple)):
                    raise TypeError("定标系数解析结果类型不正确！")
            print(f"任务完成: {desc}")
        except Exception as e:
            print(f"任务 {desc} 失败: {e}")
            break  # 如果某个任务失败，可以选择退出循环，避免后续任务因缺少必要的输入文件而失败


整体流程进度:   0%|          | 0/5 [00:00<?, ?it/s]

开始任务: 光区识别
















































































































































































































































































































































































































































































































































































整体流程进度:  20%|██        | 1/5 [00:45<03:03, 45.79s/it]

任务完成: 光区识别
开始任务: PH波段去噪


整体流程进度:  40%|████      | 2/5 [01:13<01:45, 35.17s/it]

任务完成: PH波段去噪
开始任务: RGB波段去噪


整体流程进度:  60%|██████    | 3/5 [01:22<00:46, 23.22s/it]

任务完成: RGB波段去噪
开始任务: 解析定标系数
任务完成: 解析定标系数
开始任务: 辐射校正


整体流程进度: 100%|██████████| 5/5 [01:46<00:00, 21.20s/it]

辐射校正完成
任务完成: 辐射校正
